In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

# Overview of the Data

In [2]:
%%sql
SELECT *
FROM sqlite_master
WHERE type = 'table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [3]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


# Summary Statistics (looking for any outliers)

In [4]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,7256490011,0.0,4.02


We see a few interesting things in the summary statistics:
* There's a country with a population of 0
* There's a country with a population of 7256490011 (or more than 7.2 billion people)

# Exploring Outliers

### _countries with the minimum population_

In [5]:
%%sql
SELECT MIN(population)
FROM facts;

 * sqlite:///factbook.db
Done.


MIN(population)
0


In [6]:
%%sql
SELECT name AS Country
FROM facts
WHERE population = (SELECT MIN(population)
                    FROM facts
                    );

 * sqlite:///factbook.db
Done.


Country
Antarctica


### _countries with the maximum population_

In [7]:
%%sql
SELECT MAX(population)
FROM facts;

 * sqlite:///factbook.db
Done.


MAX(population)
7256490011


In [8]:
%%sql
SELECT name AS Country
FROM facts
WHERE population = (SELECT MAX(population)
                    FROM facts
                    );

 * sqlite:///factbook.db
Done.


Country
World


* It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. 
* It also seems like the table contains a row for Antarctica, which explains the population of 0. 

* Now that we know this, we should recalculate the summary statistics we calculated earlier — this time excluding the row for the whole world.

# Exploring Average Population and Area

In [9]:
%%sql
SELECT MIN(population), MAX(population), MIN(population_growth), MAX(population_growth)
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               );

 * sqlite:///factbook.db
Done.


MIN(population),MAX(population),MIN(population_growth),MAX(population_growth)
0,1367485388,0.0,4.02


In [10]:
%%sql
SELECT AVG(population) AS average_population
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               );

 * sqlite:///factbook.db
Done.


average_population
32242666.56846473


In [11]:
%%sql
SELECT AVG(area) AS average_area
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               );

 * sqlite:///factbook.db
Done.


average_area
555093.546184739


# Finding Densely Populated Countries

Densely populated countries are defined as :
* Above-average values for population.
* Below-average values for area.

In [12]:
%%sql
SELECT name AS country, population, area, ROUND(population / CAST(area AS FLOAT),1) AS population_area_ratio
FROM facts
WHERE population > (SELECT AVG(population) AS average_population
                    FROM facts
                    WHERE name != (SELECT name AS Country
                                   FROM facts
                                   WHERE population = (SELECT MAX(population)
                                                       FROM facts
                                                       )
                                   )                    
                    )
AND area < (SELECT AVG(area) AS average_area
            FROM facts
            WHERE name != (SELECT name AS Country
                           FROM facts
                           WHERE population = (SELECT MAX(population)
                                               FROM facts
                                               )
                           )            
            )
ORDER BY population_area_ratio DESC;

 * sqlite:///factbook.db
Done.


country,population,area,population_area_ratio
Bangladesh,168957745,148460,1138.1
"Korea, South",49115196,99720,492.5
Philippines,100998376,300000,336.7
Japan,126919659,377915,335.8
Vietnam,94348835,331210,284.9
United Kingdom,64088222,243610,263.1
Germany,80854408,357022,226.5
Italy,61855120,301340,205.3
Uganda,37101745,241038,153.9
Thailand,67976405,513120,132.5


# Country with most people

In [13]:
%%sql
SELECT name AS most_populated_country, population
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               )
ORDER BY population DESC
LIMIT 1;

 * sqlite:///factbook.db
Done.


most_populated_country,population
China,1367485388


# Country with the highest growth rate

In [14]:
%%sql
SELECT name AS highest_growth_rate_country, population_growth
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               )
ORDER BY population_growth DESC
LIMIT 1;

 * sqlite:///factbook.db
Done.


highest_growth_rate_country,population_growth
South Sudan,4.02


# Countries having the highest ratios of water to land

In [15]:
%%sql
SELECT name AS country, CAST(area_water AS FLOAT) / area_land AS water_to_land_ratio
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               )
ORDER BY water_to_land_ratio DESC;

 * sqlite:///factbook.db
Done.


country,water_to_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832
Puerto Rico,0.5547914317925592
"Bahamas, The",0.3866133866133866
Guinea-Bissau,0.2846728307254623
Malawi,0.25939625850340137
Netherlands,0.22571032366565366
Uganda,0.22292237442922375
Eritrea,0.16435643564356436
Liberia,0.15623961794019933


### _countries having more water than land_

In [16]:
%%sql
SELECT name AS country, CAST(area_water AS FLOAT) / area_land AS water_to_land_ratio
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               )
AND water_to_land_ratio > 1
ORDER BY water_to_land_ratio DESC;

 * sqlite:///factbook.db
Done.


country,water_to_land_ratio
British Indian Ocean Territory,905.6666666666666
Virgin Islands,4.520231213872832


# Countries that will be adding the most people to their populations next year

In [17]:
%%sql
SELECT name AS country, ROUND((population_growth / 100) * population, 0) AS population_added_next_year
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               )
ORDER BY population_added_next_year DESC;

 * sqlite:///factbook.db
Done.


country,population_added_next_year
India,15270686.0
China,6153684.0
Nigeria,4448270.0
Pakistan,2906653.0
Ethiopia,2874562.0
Bangladesh,2703324.0
United States,2506677.0
Indonesia,2355142.0
"Congo, Democratic Republic of the",1944691.0
Philippines,1626074.0


# Countries having a higher death rate than birth rate

In [18]:
%%sql
SELECT name AS country, death_rate, birth_rate,  CAST(death_rate AS FLOAT) / birth_rate AS death_to_birth_ratio
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               )
AND death_to_birth_ratio > 1;

 * sqlite:///factbook.db
Done.


country,death_rate,birth_rate,death_to_birth_ratio
Austria,9.42,9.41,1.0010626992561105
Belarus,13.36,10.7,1.2485981308411216
Bosnia and Herzegovina,9.75,8.87,1.0992108229988726
Bulgaria,14.44,8.92,1.6188340807174888
Croatia,12.18,9.45,1.288888888888889
Czech Republic,10.34,9.63,1.0737279335410175
Estonia,12.4,10.51,1.1798287345385348
Germany,11.42,8.47,1.3482880755608027
Greece,11.09,8.66,1.2806004618937643
Hungary,12.73,9.16,1.3897379912663756


# Countries having the highest population/area ratio

In [19]:
%%sql
SELECT name AS country, population, area, ROUND(population / CAST(area AS FLOAT),1) AS population_area_ratio
FROM facts
WHERE name != (SELECT name AS Country
               FROM facts
               WHERE population = (SELECT MAX(population)
                                   FROM facts
                                   )
               )
ORDER BY population_area_ratio DESC;

 * sqlite:///factbook.db
Done.


country,population,area,population_area_ratio
Macau,592731,28,21169.0
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.3
Hong Kong,7141106,1108,6445.0
Gaza Strip,1869055,360,5191.8
Gibraltar,29258,6,4876.3
Bahrain,1346613,760,1771.9
Maldives,393253,298,1319.6
Malta,413965,316,1310.0
Bermuda,70196,54,1299.9


In [20]:
%%sql
SELECT AVG(population) AS avg_pop_top_20, AVG(area) AS avg_area_top_20, 
       (SELECT population
        FROM facts
        WHERE name = 'Bangladesh'
        ) AS bangladesh_population,
       (SELECT area
        FROM facts
        WHERE name = 'Bangladesh'
        ) AS bangladesh_area
FROM (SELECT name AS country, population, area, ROUND(population / CAST(area AS FLOAT),1) AS population_area_ratio
      FROM facts
      WHERE name != (SELECT name AS Country
                      FROM facts
                      WHERE population = (SELECT MAX(population)
                                          FROM facts
                                          )
                      )
      ORDER BY population_area_ratio DESC
      LIMIT 20
      );

 * sqlite:///factbook.db
Done.


avg_pop_top_20,avg_area_top_20,bangladesh_population,bangladesh_area
10904808.3,10070.05,168957745,148460


In [21]:
%%sql
SELECT bangladesh_population / avg_pop_top_20 AS population_comparison,
       bangladesh_area / avg_area_top_20 AS area_comparison
FROM (SELECT AVG(population) AS avg_pop_top_20, AVG(area) AS avg_area_top_20, 
              (SELECT population
              FROM facts
              WHERE name = 'Bangladesh'
              ) AS bangladesh_population,
              (SELECT area
              FROM facts
              WHERE name = 'Bangladesh'
              ) AS bangladesh_area
      FROM (SELECT name AS country, population, area, ROUND(population / CAST(area AS FLOAT),1) AS population_area_ratio
              FROM facts
              WHERE name != (SELECT name AS Country
                              FROM facts
                              WHERE population = (SELECT MAX(population)
                                                  FROM facts
                                                  )
                              )
              ORDER BY population_area_ratio DESC
              LIMIT 20
              )        
      );

 * sqlite:///factbook.db
Done.


population_comparison,area_comparison
15.493875761208933,14.742727195992076


* The top 10 countries in the tables for most densely populated countries and highest population/area ratio, we see no match.
  - It is seen that the top countries in the highest population/area ratio table all have very small areas(less than 1200 sq. kms), which massively drives up their population density(even if their populations are less - which is usually the case). Hence overall area of a country seems to be having more influence than overall population numbers.
  - Only Bangladesh seems to be present at the top (i.e. top 20) amongst countries having highest population to area ratio. This is primarily due to the extremely high number of people in the country (approx. 169 million -> approx. 15.5 times the average top 20 population) and despite having a much larger country area (148460 sq kms -> approx. 14.7 times the average area of top areas) than many top countries in the aforementioned table.
    - The ratio of these comparisons is slightly greater than 1, which is why Bangladesh is in the middle of top 20